In [3]:
import pandas as pd
import pickle
from tqdm import tqdm,tqdm_notebook
parent_path='../Data/New_Data_15-06-2020/'

In [156]:
annotated_df=pd.read_pickle(parent_path+'Fearspeech_data_final.pkl')
whatsapp_data=pd.read_csv(parent_path+'Data_text_spam_removed_v02.csv')
temp=whatsapp_data[['group_id_anonymized','phone_num_anonymized','message_text','timestamp']]
duplicateDFRow = temp[temp.duplicated()]
whatsapp_data=whatsapp_data.drop(list(duplicateDFRow.index))

from langdetect import detect
from langdetect import DetectorFactory
import parmap

DetectorFactory.seed = 0

def add_language(df):
    language= []
    for index, row in tqdm(df.iterrows(), total=len(df)):
        try:
            lang=detect(row["message_text"])
            language.append(lang)
        except:
            language.append('unk')

    df["language"] = language
    return df

n_process=30


df_split = np.array_split(whatsapp_data, n_process)
list_all = parmap.map(add_language, df_split,pm_processes=n_process,pm_pbar=True)
whatsapp_data=pd.concat(list_all,axis=0)


100%|██████████| 30/30 [20:55<00:00, 41.86s/it]


In [159]:
whatsapp_data=whatsapp_data[whatsapp_data['language'].isin(['hi','en'])]


In [157]:
import numpy as np

def Match(df, caliper = 0.05, caliper_method = "propensity", replace = False):
        
    
    # Transform the propensity scores and caliper when caliper_method is "logit" or "none"
    if caliper_method == "logit":
        propensity = log(propensity/(1-propensity))
        caliper = caliper*np.std(propensity)
    elif caliper_method == "none":
        caliper = 0
    
    # Code groups as 0 and 1
    #groups = groups == np.unique(groups)
    N = len(df)
    N1 = df[df['treatment'] == 1].index; N2 =df[df['treatment'] == 0].index
    g1, g2 = df.loc[N1]['pscore'],df.loc[N2]['pscore']
    # Check if treatment groups got flipped - the smaller should correspond to N1/g1
    if len(N1) > len(N2):
       N1, N2, g1, g2 = N2, N1, g2, g1
        
        
    # Randomly permute the smaller group to get order for matching
    morder = np.random.permutation(N1)
    matches = {}

    
    for m in morder:
        dist = abs(g1[m] - g2)
        if (dist.min() <= caliper) or not caliper:
            matches[m] = dist.idxmin()    # Potential problem: check for ties
            if not replace:
                g2 = g2.drop(matches[m])
    return (matches)

def whichMatched(matches, data, many = False, unique = False):
    ''' 
    Simple function to convert output of Matches to DataFrame of all matched observations
    Inputs:
    matches = output of Match
    data = DataFrame of covariates
    many = Boolean indicating if matching method is one-to-one or one-to-many
    unique = Boolean indicating if duplicated individuals (ie controls matched to more than one case) should be removed
    '''

    tr = matches.keys()
    if many:
        ctrl = [m for matchset in matches.values() for m in matchset]
    else:
        ctrl = matches.values()
    # need to remove duplicate rows, which may occur in matching with replacement
    temp = pd.concat([data.loc[tr], data.loc[ctrl]])
    if unique == True:
        return temp.groupby(temp.index).first()
    else:
        return temp


In [158]:
from datetime import datetime


def convert(year,month,date):
    return int(datetime(year, month, date, 0, 0, 0).timestamp()*1000)

def convert_reverse(timestamp):
    dt_object = datetime.fromtimestamp(timestamp/1000)
    print("dt_object =", dt_object)
    return dt_object

In [160]:
convert_reverse(1530948979856)

dt_object = 2018-07-07 13:06:19.856000


datetime.datetime(2018, 7, 7, 13, 6, 19, 856000)

In [161]:
whatsapp_data=whatsapp_data.sort_values('timestamp')

In [162]:
whatsapp_data.head(5)

,group_id_anonymized,phone_num_anonymized,message_text,timestamp,media_type,media_url,forwarded,orig_index,language,only_urls,is_spam
2163788,8982,2148,What,1530949216000,None,None,0,5500170,en,0,0
2465534,9860,247483,#मंज़िल की तलाश में सड़को को #रौंद रहे है.... ...,1532187844000,None,None,64,6243091,hi,0,0
2471762,252,1287,इस मादर चोद को इतना शेयर करो कि यह पकडा जाय यह...,1532187995000,None,None,1,6257572,hi,0,0
2409489,4016,244632,Follow this link to join my WhatsApp group: ht...,1532188368000,None,https://chat.whatsapp.com,64,6099306,en,0,0
2465543,9860,247483,चलते रहो #पथ पर ...चलने में #माहिर बन जाओगे......,1532188521000,None,None,64,6243100,hi,0,0


In [163]:
all_fear_speech_index=[]
all_normal_index=[]

count_fearspeech=0
count_normal=0

for index,row in tqdm(annotated_df.iterrows(),total=len(annotated_df)):
    if(row['one_fear_speech']==1):
        count_fearspeech+=1
        all_fear_speech_index+=row['repeated']
    elif(row['one_fear_speech']==0):
        count_normal+=1
        all_normal_index+=row['repeated']

data_fear_speech=whatsapp_data[whatsapp_data['orig_index'].isin(all_fear_speech_index)]
data_normal=whatsapp_data[whatsapp_data['orig_index'].isin(all_normal_index)]


100%|██████████| 4782/4782 [00:00<00:00, 8509.59it/s]


In [164]:
import numpy as np

tuples_list=[]

for user in tqdm_notebook(whatsapp_data.phone_num_anonymized.unique(),total=whatsapp_data.phone_num_anonymized.nunique()):
    temp={}
    temp['user_id']=user
    user_data=whatsapp_data[whatsapp_data['phone_num_anonymized']==user]
    if(user in set(data_fear_speech.phone_num_anonymized.unique())):
        temp['treatment']=1
    else:
        temp['treatment']=0
        
   
    temp['num_groups']=user_data['group_id_anonymized'].nunique()
    count=0
    list_message_present=[]
    for year in [2018,2019]:
        for month in range(1,13):
            timestamp=convert(year,month,1)
            count+=1
            temp_data=user_data[user_data['timestamp'].between(timestamp, timestamp+30*24*60*60*1000, inclusive=True)]
            if(len(temp_data)>0):
                try:
                    start=temp['start_date']
                except KeyError:
                    temp['start_date']=count
                list_message_present.append(len(temp_data))
    if(len(list_message_present)>0):
        temp['mean_messages']=np.mean(list_message_present)
        temp['stdev_messages']=np.std(list_message_present)
        temp['month_present']=len(list_message_present)
        tuples_list.append(temp)         


/home/punyajoy/.conda/envs/punyajoy_gpu/lib/python3.7/site-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


In [165]:
df = pd.DataFrame(tuples_list) 

In [166]:
df

,user_id,treatment,num_groups,start_date,mean_messages,stdev_messages,month_present
0,2148,0,154,7,24.571429,21.181914,14
1,247483,0,1,7,10.000000,1.632993,3
2,1287,1,1,7,4.333333,3.399346,3
3,244632,0,2,7,5.000000,3.674235,4
4,242857,0,1,7,32.333333,24.115463,3
...,...,...,...,...,...,...,...
108692,23730,0,1,20,1.000000,0.000000,1
108693,174891,0,1,20,1.000000,0.000000,1
108694,174892,0,1,20,20.000000,0.000000,1
108695,174080,0,1,20,1.000000,0.000000,1


In [194]:
from pscore_match.pscore import PropensityScore
#from pscore_match.match import Match, whichMatched
from scipy.stats import gaussian_kde
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


treatment = np.array(df.treatment)
cov_list = ['num_groups','start_date','mean_messages','stdev_messages','month_present']
#cov_list = ['start_date','month_present']

covariates = df[cov_list]

pscore = PropensityScore(treatment, covariates).compute()
df['pscore']=pscore
pairs = Match(df,caliper = 0.5)
data_matched=whichMatched(pairs, df,many = False, unique = False)
# pairs = Match(treatment, pscore)
# pairs.create(method='one-to-one',caliper=0.001,replace=False)
# data_matched = whichMatched(pairs, pd.DataFrame({'pscore': pscore, 'treatment' :treatment, 'user_id':df.user_id}))


In [195]:
treatment=np.array(data_matched[data_matched['treatment']==1].pscore)
control=np.array(data_matched[data_matched['treatment']==0].pscore)

In [196]:
data_fear_speech.phone_num_anonymized.nunique()

2933

In [197]:
from numpy.random import seed
from numpy.random import randn
from scipy.stats import mannwhitneyu
# seed the random number generator
seed(1)
# generate two independent samples
# compare samples
stat, p = mannwhitneyu(treatment, control)
print('Statistics=%.3f, p=%.3f' % (stat, p))
# interpret
alpha = 0.05
if p > alpha:
	print('Same distribution (fail to reject H0)')
else:
	print('Different distribution (reject H0)')

Statistics=4298233.000, p=0.500
Same distribution (fail to reject H0)


In [198]:
len(data_matched)/2

2932.0

In [26]:
matched_groups=[]
for index,row in data_matched.iterrows():
    if(row['treatment']==1):
        matched_groups.append([int(row['user_id']),'Fear speech'])
    else:
        matched_groups.append([int(row['user_id']),'Non fear speech'])
    

In [27]:
df_group_matched=pd.DataFrame(matched_groups,columns=['user_id','annotation'])

In [28]:
df_group_matched.to_csv(parent_path+'final_user_selection_matched.csv',index=False)

### Group matching


In [29]:
from collections import OrderedDict 
group_dict={}
for group in data_fear_speech.group_id_anonymized.unique():
    temp=data_fear_speech[data_fear_speech['group_id_anonymized']==group]
    group_dict[group]=len(temp)
dict1 = sorted(group_dict.items(),key=lambda item: item[1],reverse=True)

selected_groups=[ele[0] for ele in dict1[0:100]]
len(selected_groups)

100

In [145]:
import numpy as np

tuples_list_grp=[]
for user in tqdm_notebook(whatsapp_data.group_id_anonymized.unique(),total=whatsapp_data.group_id_anonymized.nunique()):
    temp={}
    temp['group_id']=user
    user_data=whatsapp_data[whatsapp_data['group_id_anonymized']==user]
    if(user in data_fear_speech.group_id_anonymized.unique()):
        temp['treatment']=1
    else:
        temp['treatment']=0
        
   
    temp['num_users']=user_data['phone_num_anonymized'].nunique()
    count=0
    list_message_present=[]
    for year in [2018,2019]:
        for month in range(1,13):
            timestamp=convert(year,month,1)
            count+=1
            temp_data=user_data[user_data['timestamp'].between(timestamp, timestamp+30*24*60*60*1000, inclusive=True)]
            if(len(temp_data)>0):
                try:
                    start=temp['start_date']
                except KeyError:
                    temp['start_date']=count
                list_message_present.append(len(temp_data))
    if(len(list_message_present)>0):
        temp['mean_messages']= len(user_data)/temp['num_users']
        temp['stdev_messages']=np.std(list_message_present)
        temp['month_present']=len(list_message_present)
        tuples_list_grp.append(temp)         



/home/punyajoy/.conda/envs/punyajoy_gpu/lib/python3.7/site-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


In [147]:
df_groups = pd.DataFrame(tuples_list_grp) 

In [148]:
df_groups = df_groups[df_groups['treatment']!=-1]

In [149]:
len(df_groups[df_groups['treatment']==1])

950

In [150]:
from pscore_match.pscore import PropensityScore
#from pscore_match.match import Match, whichMatched
from scipy.stats import gaussian_kde
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


treatment = np.array(df_groups.treatment)
cov_list = ['start_date','num_users','month_present','mean_messages']
#cov_list = ['start_date','month_present']

covariates = df_groups[cov_list]

pscore = PropensityScore(treatment, covariates).compute()
df_groups['pscore']=pscore
pairs = Match(df_groups,caliper = 1)
data_matched=whichMatched(pairs, df_groups, many = False, unique = False)
# pairs = Match(treatment, pscore)
# pairs.create(method='one-to-one',caliper=0.001,replace=False)
# data_matched = whichMatched(pairs, pd.DataFrame({'pscore': pscore, 'treatment' :treatment, 'user_id':df.user_id}))


In [151]:
treatment=np.array(data_matched[data_matched['treatment']==1].pscore)
control=np.array(data_matched[data_matched['treatment']==0].pscore)


print(data_fear_speech.group_id_anonymized.nunique())
print(len(treatment))

950
950


In [152]:
df_groups[df_groups['treatment']==1]

,group_id,treatment,num_users,start_date,mean_messages,stdev_messages,month_present,pscore
1,9860,1,188,7,10.952128,162.887913,6,0.354408
2,252,1,175,7,5.360000,67.728420,12,0.625226
5,4016,1,331,7,18.531722,501.828196,13,0.917193
9,9337,1,78,7,11.871795,53.705214,6,0.143876
11,4872,1,5,7,834.800000,177.229717,14,0.355488
...,...,...,...,...,...,...,...,...
5517,8800,1,10,19,3.700000,11.500000,2,0.081120
5522,8797,1,10,19,1.900000,8.500000,2,0.081070
5540,8810,1,14,20,5.000000,0.000000,1,0.074788
5542,8812,1,18,20,3.777778,0.000000,1,0.077790


In [153]:
from scipy.stats import pearsonr
# seed random number generator
seed(1)
# prepare data
data1 = 20 * randn(1000) + 100
data2 = data1 + (10 * randn(1000) + 50)
# calculate Pearson's correlation
corr, _ = pearsonr(df_groups['start_date'], df_groups['month_present'])
print('Pearsons correlation: %.3f' % corr)

Pearsons correlation: -0.484


In [154]:
from numpy.random import seed
from numpy.random import randn
from scipy.stats import mannwhitneyu
# seed the random number generator
seed(2)
# generate two independent samples
# compare samples
stat, p = mannwhitneyu(treatment, control)
print('Statistics=%.3f, p=%.3f' % (stat, p))
# interpret
alpha = 0.005
if p > alpha:
	print('Same distribution (fail to reject H0)')
else:
	print('Different distribution (reject H0)')

Statistics=450665.500, p=0.481
Same distribution (fail to reject H0)


In [94]:
matched_groups=[]
for index,row in data_matched.iterrows():
    if(row['treatment']==1):
        matched_groups.append([int(row['group_id']),'Fear speech'])
    else:
        matched_groups.append([int(row['group_id']),'Non fear speech'])
    

In [133]:
df_group_matched=pd.DataFrame(matched_groups,columns=['grp_id','annotation'])

In [134]:
df_group_matched.to_csv(parent_path+'final_group_selection_matched.csv',index=False)

In [97]:
df_group_matched

,grp_id,annotation
0,9851,Fear speech
1,960,Fear speech
2,4057,Fear speech
3,936,Fear speech
4,838,Fear speech
...,...,...
167,1134,Non fear speech
168,81,Non fear speech
169,2656,Non fear speech
170,5071,Non fear speech


In [135]:
len(treatment)

896